In [5]:
import torch 
import torch.nn as nn
import torch.utils.checkpoint as checkpoint
from timm.models.layers import DropPath, to_2tuple, trunc_normal_

In [ ]:
class MLP(nn.Module):
    def __init__(self,in_features,hidden_features=None,out_features=None,act_layer=nn.GELU,drop=0):
        super().__init__()
        self.out_features=out_features or in_features
        self.hidden_features=hidden_features
        self.fc1=nn.Linear(in_features,hidden_features)
        self.act=act_layer()
        self.drop=nn.Dropout(drop)
    
    def forward(self,x):
        x=self.fc1(x)
        x=self.act(x)
        x=self.drop(x)
        x=self.fc1(x)
        x=self.drop(x)

        return x

In [8]:
def window_partion(x,window_size):
    '''
    Args:
        x:(B,H,W,C)
        window_size (int): window size

    Returns:
        windows:(num_windows*B, window_size, window_size, C)
    '''
    B,H,W,C=x.shape
    x=x.view(B,H//window_size,window_size,W//window_size,window_size,C)
    windows=x.permute(0,1,3,2,4,5).contiguous().view(-1,window_size,C)
    
    return windows

In [6]:
x=torch.randn(32,8,8,48)
a=window_partion(x,4)
a.shape 

NameError: name 'window_partion' is not defined

In [ ]:
def window_reverse(windows,window_size,H,W):
    '''
    Args:
        windows:(num_windows*B,window_size,window_size,C)
        H (int): height of image
        W (int): width of image
    
    Returns:
        x: (B,H,W,C)
    '''

    B=int(windows.shape[0]/(H*W/window_size/window_size))
    x=windows.view(B,H//window_size,window_size,W//window_size,window_size,-1)
    x=x.permute(0,1,3,2,4,5).contiguous.view(B,H,W,-1)

In [ ]:
class WindowAttention(nn.Module):
    '''
    Windows based multi head self attention (W-MSA) module with relative position bias
    It supports both of shifted and non-shifted window
    
    Args:
        dim (int): Number of input channels
        window_size (tuple[int]): The height of width of the windows
        num_heads (int): Number of attention heads
        qkv_bias (bool, optional): If true, add a learnable bias to query,key,value. Default: True
        qk_scale (float| None, optional): Override default qk sclale of head**-0.5 if set
        attn_drop (float, optional): Dropout ratio of attention weight. Default 0.0
        proj_drop (float, optional): Dropout ratio of output. Default 0.0         
    '''

    def __init__(self,dim,window_size,num_heads,qkv_bias=True,qk_scale=None,attn_drop=0.,proj_drop=0.):
        super().__init__()
        self.dim=dim
        self.window_size=window_size # Wh, Ww
        self.num_heads=num_heads
        head_dim=dim//num_heads
        self.scale=qk_scale or head_dim**-0.5

        # define a parameter table of relative position bias
        self.relative_position_bias_table=nn.Parameter(
            torch.zeros((2*self.window_size-1)*(2*window_size[1]-1,num_heads))) # 2*Wh-1 * 2*Ww-1, nH
        
        #get pair-wise


In [7]:
a=nn.Parameter(
            torch.zeros((2 * 32 - 1) * (2 * 32 - 1), 8))
a.shape

torch.Size([3969, 8])